In [1]:
"""
Created on Wed Mar 24 16:18 2020

This is a script to cut out the T and S and average them over the domains of 10, 25, 50, 100 km 
in front of the ice shelf and an offshore domain

@author: Clara Burgard
"""

'\nCreated on Wed Mar 24 16:18 2020\n\nThis is a script to cut out the T and S and average them over the domains of 10, 25, 50, 100 km \nin front of the ice shelf and an offshore domain\n\n@author: Clara Burgard\n'

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
#import assess_param_funcs.useful_functions as uf
#import assess_param_funcs.T_S_profile_functions as tspf
#import assess_param_funcs.melt_functions as meltf
#import assess_param_funcs.box_functions as bf
import multimelt.useful_functions as uf
import multimelt.T_S_profile_functions as tspf
import multimelt.melt_functions as meltf
import multimelt.box_functions as bf

from scipy.spatial import cKDTree


import itertools

import distributed
import glob

READ IN THE DATA

In [3]:
nemo_run = 'OPM018'

if nemo_run == 'OPM006':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM021':
    yy_start = 1989
    yy_end = 2018
elif nemo_run == 'OPM016' or nemo_run == 'OPM018':
    yy_start = 1980
    yy_end = 2008

In [4]:
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
#inputpath_profiles='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_profiles='../../../data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
inputpath_isf='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

outputpath_profiles = '../../../data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

PREPARE MASK AROUND FRONT (TO RUN WITHOUT DASK!)

In [5]:
T_S_ocean_1980 = xr.open_dataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc')
file_isf_orig = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf = file_isf_orig.sel(Nisf=nonnan_Nisf)

In [6]:
file_mask_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_Ant_stereo.nc')
file_mask = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim).squeeze().drop('time')

In [7]:
lon = file_isf['longitude']
lat = file_isf['latitude']

In [8]:
ocean = np.isfinite(T_S_ocean_1980['theta_ocean'].isel(time=0,depth=0)).drop('time').drop('depth')
# only points below 1500 m
offshore = file_mask['bathy_metry'] > 1500 # .drop('lon').drop('lat')
# only points above 1500 m
contshelf = file_mask['bathy_metry'] <= 1500 # .drop('lon').drop('lat')

In [9]:
#mask_domains = (ocean & contshelf).load() #<= checked if it does what it should and it does! :)
#mask_domains = (ocean).load()
# NB: 5.0 x 1.75 is the effective resolution at 70S for a model of 1 degree resolution in longitude (assuming 5 delta X and a Mercator grid)
mask_domains = xr.DataArray([(ocean & contshelf), (ocean & offshore)],
                            dims={'profile_domain': ['close_cont_shelf','offshore'], 'y': contshelf.y, 'x': contshelf.x}).load()

lon_box = xr.DataArray(np.array([10.0, 10.0]), coords=[('profile_domain', ['close_cont_shelf','offshore'])])
lat_box = xr.DataArray(np.array([3.5, 3.5]), coords=[('profile_domain', ['close_cont_shelf','offshore'])])  

In [10]:
close_region_around_isf_mask = tspf.mask_boxes_around_IF_new(lon, lat, mask_domains, 
                                    file_isf['front_min_lon'], file_isf['front_max_lon'], 
                                    file_isf['front_min_lat'], file_isf['front_max_lat'],  
                                    lon_box, lat_box, 
                                    file_isf['isf_name'])

In [11]:
def distance_isf_points_from_line_small_domain(isf_points_da,line_points_da):

    """
    Compute the distance between ice shelf points and a line.
    
    This function computes the distance between ice shelf points and a line. This line can be the grounding
    line or the ice shelf front.
    
    Parameters
    ----------
    whole_domain : xarray.DataArray
        ice-shelf mask - all ice shelves are represented by a number, all other points (ocean, land) set to nan
    isf_points_da : xarray.DataArray
        array containing only points from one ice shelf
    line_points_da : xarray.DataArray
        mask representing the grounding line or ice shelf front mask corresponding to the ice shelf selected in ``isf_points_da``
        
    Returns
    -------
    xr_dist_to_line : xarray.DataArray
        distance of the each ice shelf point to the given line of interest
    """

    # add a common dimension 'grid' along which to stack
    stacked_isf_points = isf_points_da.stack(grid=['y', 'x'])
    stacked_line = line_points_da.stack(grid=['y', 'x'])

    # remove nans
    filtered_isf_points = stacked_isf_points[stacked_isf_points>0]
    filtered_line = stacked_line[stacked_line>0]

    # write out the y,x pairs behind the dimension 'grid'
    grid_isf_points = filtered_isf_points.indexes['grid'].to_frame().values.astype(float)
    grid_line = filtered_line.indexes['grid'].to_frame().values.astype(float)

    # create tree to line and compute distance
    tree_line = cKDTree(grid_line)
    dist_yx_to_line, _ = tree_line.query(grid_isf_points)

    # add the coordinates of the previous variables
    xr_dist_to_line = filtered_isf_points.copy(data=dist_yx_to_line)
    # put 1D array back into the format of the grid and put away the 'grid' dimension
    xr_dist_to_line = xr_dist_to_line.unstack('grid')

    return xr_dist_to_line


In [12]:
dist_list = [ ]
for kisf in tqdm(file_isf['Nisf']):
        
        if (file_isf['IF_mask']==kisf).sum() > 0:
            region_to_cut_out = close_region_around_isf_mask.sel(profile_domain='close_cont_shelf').sel(Nisf=kisf)
            region_to_cut_out = region_to_cut_out.where(region_to_cut_out > 0, drop=True)
            IF_region = file_isf['IF_mask'].where(file_isf['IF_mask']==kisf, drop=True)

            #dist_from_front = tspf.distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
            dist_from_front = distance_isf_points_from_line_small_domain(region_to_cut_out,IF_region)
            dist_list.append(dist_from_front)
    
dist_all = xr.concat(dist_list, dim='Nisf').reindex_like(file_isf)

100%|███████████████████████████████████| 122/122 [00:12<00:00, 10.12it/s]


In [13]:
dist_all.to_dataset(name='dist_from_front').to_netcdf(outputpath_profiles+'dist_to_ice_front_only_contshelf.nc')
#dist_all.to_dataset(name='dist_from_front').to_netcdf(outputpath_profiles+'dist_to_ice_front_whole_domain.nc')

In [14]:
close_region_around_isf_mask.sel(profile_domain='offshore').to_dataset(name='mask').to_netcdf(outputpath_profiles+'mask_offshore.nc')

COMPUTING THE MEAN PROFILES (TO RUN WITH DASK)

In [15]:
client = distributed.Client(n_workers=24, dashboard_address=':8796', local_directory='/tmp', memory_limit='6GB')

In [16]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8796/status,
Dashboard: http://127.0.0.1:8796/status,Workers: 24
Total threads: 24,Total memory: 134.11 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44925,Workers: 24
Dashboard: http://127.0.0.1:8796/status,Total threads: 24
Started: Just now,Total memory: 134.11 GiB
Comm: tcp://127.0.0.1:38823,Total threads: 1
Dashboard: http://127.0.0.1:37051/status,Memory: 5.59 GiB
Nanny: tcp://127.0.0.1:44381,


CONTINENTAL SHELF

In [17]:
#bbox_da = xr.DataArray(np.array([10000., 25000., 50000., 100000.]), coords=[('dist_from_front', [10,25,50,100])])
bbox_da = 50000

If workers don't die (with 12 cores, took approx 1hour), if workers die, divide work by years

In [18]:
all_in_one = False # False if worker die, True if workers don't die
if all_in_one:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc',chunks={'x': 50, 'y': 50})
    T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', combine='nested',concat_dim='time', chunks={'x': 50, 'y': 50, 'depth': 50}, parallel=True)
    #T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_1990.nc',chunks={'x': 50, 'y': 50, 'depth': 50})
    T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc',chunks={'x': 50, 'y': 50, 'depth': 50})
else:
    dist_to_front_file = xr.open_mfdataset(inputpath_profiles+'dist_to_ice_front_only_contshelf.nc',chunks={'x': 100, 'y': 100})
    T_S_ocean_files = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_*.nc', combine='nested', concat_dim='time', chunks={'x': 100, 'y': 100, 'depth': 50}, parallel=True)
    #T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_1990.nc',chunks={'x': 100, 'y': 100, 'depth': 50})
    T_S_ocean_1980 = xr.open_mfdataset(inputpath_profiles+'T_S_theta_ocean_corrected_2000.nc',chunks={'x': 100, 'y': 100, 'depth': 50})
dist_to_front = dist_to_front_file['dist_from_front']

Prepare sum

In [19]:
mask_km = dist_to_front <= bbox_da

In [20]:
ds_sum = (T_S_ocean_files * mask_km).sum(['x','y'])

In [21]:
if all_in_one:
    ds_sum = ds_sum.load()
    ds_sum.to_netcdf(outputpath_profiles+'ds_sum_for_mean_contshelf.nc')
else:
    yearly_datasets = list(tspf.split_by_chunks(ds_sum.unify_chunks(),'time'))
    paths = [tspf.create_filepath(ds, 'ds_sum_for_mean_contshelf', outputpath_profiles, ds.time[0].values) for ds in yearly_datasets]
    xr.save_mfdataset(datasets=yearly_datasets, paths=paths)

Prepare number of points by which you divide

In [22]:
if all_in_one:
    ds_sum = xr.open_mfdataset(inputpath_profiles+'ds_sum_for_mean_contshelf.nc')
else:
    ds_sum = xr.open_mfdataset(inputpath_profiles+'ds_sum_for_mean_contshelf_*.nc', combine='nested', concat_dim='time', parallel=True).drop('profile_domain')

In [23]:
mask_depth = T_S_ocean_1980['salinity_ocean'].squeeze().drop('time') >0
mask_all = mask_km & mask_depth

In [24]:
mask_sum = mask_all.sum(['x','y'])

In [25]:
mask_sum = mask_sum.load()

Make the mean

In [26]:
ds_mean = ds_sum/mask_sum

In [27]:
ds_mean.drop('profile_domain')

<xarray.Dataset>
Dimensions:         (time: 29, depth: 121, Nisf: 122)
Coordinates:
  * time            (time) int64 1980 1981 1982 1983 ... 2005 2006 2007 2008
  * depth           (depth) float64 0.492 1.526 2.638 ... 5.697e+03 5.902e+03
  * Nisf            (Nisf) int64 4 5 6 8 10 11 12 ... 205 210 211 213 214 216
Data variables:
    theta_ocean     (time, depth, Nisf) float64 dask.array<chunksize=(1, 121, 122), meta=np.ndarray>
    salinity_ocean  (time, depth, Nisf) float64 dask.array<chunksize=(1, 121, 122), meta=np.ndarray>

In [28]:
ds_mean = ds_mean.drop('profile_domain').rename({'dist_from_front': 'profile_domain'})

ValueError: cannot rename 'dist_from_front' because it is not a variable or dimension in this dataset

In [29]:
ds_mean.to_netcdf(outputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_1980-2018.nc')

OFFSHORE PROFILES

In [31]:
T_S_ocean_files = xr.open_mfdataset(outputpath_profiles+'T_S_theta_ocean_corrected_*.nc', combine='nested', concat_dim='time', chunks={'x': 50, 'y': 50, 'depth': 50}, parallel=True)
T_S_ocean_1980 = xr.open_mfdataset(outputpath_profiles+'T_S_theta_ocean_corrected_2000.nc',chunks={'x': 50, 'y': 50, 'depth': 50})

In [32]:
mask_offshore_file = xr.open_mfdataset(inputpath_profiles+'mask_offshore.nc')
mask_offshore = mask_offshore_file['mask'].drop('profile_domain')
mask_depth = T_S_ocean_1980['salinity_ocean'].squeeze().drop('time') >0
mask_all_offshore = mask_offshore & mask_depth

In [33]:
ds_sum_offshore = (T_S_ocean_files * mask_offshore).sum(['x','y'])
ds_sum_offshore['profile_domain'] = np.array([1000])

In [34]:
ds_sum_offshore = ds_sum_offshore.load()
ds_sum_offshore.to_netcdf(outputpath_profiles+'ds_sum_for_mean_offshore.nc')

/home/ockendeh/miniconda3/envs/nnets_py38/lib/python3.8/site-packages/distributed/client.py:3106: UserWarning: Sending large graph of size 33.06 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [35]:
mask_sum_offshore = mask_all_offshore.sum(['x','y'])

In [36]:
mask_sum_offshore = mask_sum_offshore.load()

In [37]:
ds_mean_offshore = ds_sum_offshore/mask_sum_offshore

In [38]:
ds_mean_offshore.to_netcdf(outputpath_profiles+'T_S_mean_prof_corrected_km_offshore_1980-2018.nc')

COMBINE BOTH

In [39]:
ds_mean_offshore = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_offshore_1980-2018.nc')
ds_mean = xr.open_dataset(outputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_1980-2018.nc')#.drop('profile_domain').rename({'dist_from_front':'profile_domain'})

In [40]:
ds_mean_both = xr.concat([ds_mean, ds_mean_offshore], dim='profile_domain')
ds_mean_both.to_netcdf(outputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc')